In [9]:
test = False
with open('poppy.txt') as _:
    poppy = _.read()
print(poppy)
print('NIC CAGE FANTASY DRAFT')

░░░░░░░░░░░░░░▄▄▄▄▄▄▄▄▄▄▄▄░░░░░░░░░░░░░░
░░░░░░░░░░░░▄████████████████▄░░░░░░░░░░
░░░░░░░░░░▄██▀░░░░░░░▀▀████████▄░░░░░░░░
░░░░░░░░░▄█▀░░░░░░░░░░░░░▀▀██████▄░░░░░░
░░░░░░░░░███▄░░░░░░░░░░░░░░░▀██████░░░░░
░░░░░░░░▄░░▀▀█░░░░░░░░░░░░░░░░██████░░░░
░░░░░░░█▄██▀▄░░░░░▄███▄▄░░░░░░███████░░░
░░░░░░▄▀▀▀██▀░░░░░▄▄▄░░▀█░░░░█████████░░
░░░░░▄▀░░░░▄▀░▄░░█▄██▀▄░░░░░██████████░░
░░░░░█░░░░▀░░░█░░░▀▀▀▀▀░░░░░██████████▄░
░░░░░░░▄█▄░░░░░▄░░░░░░░░░░░░██████████▀░
░░░░░░█▀░░░░▀▀░░░░░░░░░░░░░███▀███████░░
░░░▄▄░▀░▄░░░░░░░░░░░░░░░░░░▀░░░██████░░░
██████░░█▄█▀░▄░░██░░░░░░░░░░░█▄█████▀░░░
██████░░░▀████▀░▀░░░░░░░░░░░▄▀█████████▄
██████░░░░░░░░░░░░░░░░░░░░▀▄████████████
██████░░▄░░░░░░░░░░░░░▄░░░██████████████
██████░░░░░░░░░░░░░▄█▀░░▄███████████████
███████▄▄░░░░░░░░░▀░░░▄▀▄███████████████

NIC CAGE FANTASY DRAFT


## Draft factors
The fantasy draft will factor 3 inputs in listed in decreasing influence:
1) Cage Factor (Ranges from 0-10)  
2) Audience Score from Rotten Tomatoes (Ranges from 0-100, scaled to be 0-10)  
3) Critic Score from Rotten Tomatoes (Ranges from 0-100, scaled to be 0-10)  

Therefore, the max *unweighted* influencer score one could achieve would be 30, and the minimum would be 0.  A 30 is not possible, as maximum Cage factor tends to upset the critics.  As noted above, the scores will be weighted to heavily bias Cage Factor, then Audience score, and finally critic score.

Cage factor will influence draft order heavily.  While no one man can truly set the Cage Factor of even a single scene of a single movie, I will make an approximation.  Select your movie accordingly.  Bribes encouraged.

Seed for the random number generator will be the number of characters in Nic Cage's birth name, Nicolas Kim Coppola.

In [10]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
# Seed the random number generator for repeatability
blesscage = 'Nicolas Kim Coppola'
np.random.seed(len(blesscage))

In [11]:
# Pull in everyone's movie choices & Cage Factors
cage_factors = pd.read_csv('cagefactor.csv')
# Testing purposes only - filling cage factor with random number
if test:
    cage_factors['Cage Factor'] = np.random.randint(0, 10, cage_factors.shape[0])
cage_factors

,Name,Title,Cage Factor
0,Mike Georgiadis,8MM,NaN
1,Matt Wieck,National Treasure: Book of Secrets,7.0
2,Patrick Samuels,Raising Arizona,6.0
3,Mike Diaz,8MM,NaN
4,Josh Underhill,Vampire's Kiss,10.0
5,Zach Siefring,SNL Weekend Update,5.0
6,Spencer Parry,Spider-Man: Into the Spider-Verse,5.0
7,Rob Dillard,The Croods,5.0
8,Tim O'Hara,The Wicker Man,10.0
9,Tony Lucisano,Con Air,7.0


In [4]:
# Pull in Movie List
cage_movies = pd.read_csv('Cage Scores.csv')
cage_movies = cage_movies[['Title','Critics','Audience']]
# Scale Critic and Audience scores to range from 0-10
cage_movies['Critics']=cage_movies['Critics']/10
cage_movies['Audience']=cage_movies['Audience']/10
cage_movies.head()

,Title,Critics,Audience
0,The Unbearable Weight of Massive Talent,0.0,0.0
1,Willy's Wonderland,6.1,7.0
2,Prisoners of the Ghostland,7.6,0.0
3,Jiu Jitsu,2.8,6.5
4,The Croods: A New Age,7.7,9.4


In [5]:
# To accomodate Zach's weekend update choice, need to insert SNL as a separate row
# As agreed on over text, will use the average critic score for SNL since there is no critic score.
# audience score (from RT) is 44%
# Create Row to append to cage_movies
snl=pd.DataFrame()
snl['Title']=['SNL Weekend Update']
snl['Critics']=[cage_movies['Critics'].mean()]
snl['Audience']=4.4

# Append to list of cage movies
cage_movies = pd.concat([cage_movies,snl]).reset_index(drop=True)
cage_movies.tail()

,Title,Critics,Audience
103,Birdy,8.300000,8.3
104,Rumble Fish,7.400000,8.0
105,Valley Girl,8.300000,7.2
106,Fast Times at Ridgemont High,7.700000,8.0
107,SNL Weekend Update,4.280374,4.4


In [6]:
# merge together - check joins
cage_data= cage_factors.merge(right=cage_movies, 
                              how='left',
                              on = 'Title',
                              indicator=True)

if not (cage_data['_merge']=='both').all():
    print(cage_data[cage_data['_merge']!='both'])
    raise LookupError('Movies not matching between source files')

cage_data = cage_data.drop('_merge', axis=1)

cage_data

,Name,Title,Cage Factor,Critics,Audience
0,Mike Georgiadis,8MM,5,2.300000,5.2
1,Matt Wieck,National Treasure: Book of Secrets,2,3.600000,6.7
2,Patrick Samuels,Raising Arizona,8,9.100000,8.5
3,Mike Diaz,8MM,3,2.300000,5.2
4,Josh Underhill,Vampire's Kiss,4,6.100000,5.3
5,Zach Siefring,SNL Weekend Update,2,4.280374,4.4
6,Spencer Parry,Spider-Man: Into the Spider-Verse,8,9.700000,9.3
7,Rob Dillard,The Croods,6,7.200000,7.7
8,Tim O'Hara,The Wicker Man,2,1.500000,1.7
9,Tony Lucisano,Con Air,5,5.600000,7.5


In [7]:
# calculate the unweighted and weighted Cage Scores
# Weights were chosen so that both the weighted and unweighted scores range between 0 and 30
weights = {'Cage Factor':2, 'Audience':0.75, 'Critics':0.25}

# Calculate individual weighted scores
cage_data['Cage Factor-Weighted'] = cage_data['Cage Factor']*weights['Cage Factor']
cage_data['Audience-Weighted'] = cage_data['Audience']*weights['Audience']
cage_data['Critics-Weighted'] = cage_data['Critics']*weights['Critics']

# Calculate overall weighted and unweighted scores
cage_data['Overall Score-Unweighted'] = cage_data['Cage Factor'] + cage_data['Critics'] + cage_data['Audience']
cage_data['Overall Score-Weighted'] = cage_data['Cage Factor-Weighted'] + cage_data['Critics-Weighted'] + cage_data['Audience-Weighted']

# Calculate probability for np.random.choice
cage_data['Probability'] = cage_data['Overall Score-Weighted']/(cage_data['Overall Score-Weighted'].sum())

cage_data

,Name,Title,Cage Factor,Critics,Audience,Cage Factor-Weighted,Audience-Weighted,Critics-Weighted,Overall Score-Unweighted,Overall Score-Weighted,Probability
0,Mike Georgiadis,8MM,5,2.300000,5.2,10,3.900,0.575000,12.500000,14.475000,0.080486
1,Matt Wieck,National Treasure: Book of Secrets,2,3.600000,6.7,4,5.025,0.900000,12.300000,9.925000,0.055186
2,Patrick Samuels,Raising Arizona,8,9.100000,8.5,16,6.375,2.275000,25.600000,24.650000,0.137062
3,Mike Diaz,8MM,3,2.300000,5.2,6,3.900,0.575000,10.500000,10.475000,0.058245
4,Josh Underhill,Vampire's Kiss,4,6.100000,5.3,8,3.975,1.525000,15.400000,13.500000,0.075065
5,Zach Siefring,SNL Weekend Update,2,4.280374,4.4,4,3.300,1.070093,10.680374,8.370093,0.046541
6,Spencer Parry,Spider-Man: Into the Spider-Verse,8,9.700000,9.3,16,6.975,2.425000,27.000000,25.400000,0.141233
7,Rob Dillard,The Croods,6,7.200000,7.7,12,5.775,1.800000,20.900000,19.575000,0.108844
8,Tim O'Hara,The Wicker Man,2,1.500000,1.7,4,1.275,0.375000,5.200000,5.650000,0.031416
9,Tony Lucisano,Con Air,5,5.600000,7.5,10,5.625,1.400000,18.100000,17.025000,0.094665


In [8]:
names2choose = cage_data['Name']
probabilities = cage_data['Probability']
numberofchoices = len(cage_data)
draft_order = np.random.choice(a = names2choose,
                              size=numberofchoices,
                              replace=False,
                              p=probabilities)
draft_order

array(['Jim Jennings', 'Rob Dillard', 'Eric Nesi', 'Spencer Parry',
       'Mike Georgiadis', "Tim O'Hara", 'Patrick Samuels',
       'Zach Siefring', 'Josh Underhill ', 'Matt Wieck', 'Tony Lucisano',
       'Mike Diaz'], dtype=object)